# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [6]:
#You may add additional import if you want
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time

In [7]:
%matplotlib inline

In [8]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1
5,1,1,44,43,41,41,37,29,28.356400,6.935636,2.305771,0.323724,0.000000,0.000000,0.000000,0.000000,0.502831,0.126741,0,1
6,1,0,29,29,29,27,25,16,15.448398,9.113819,1.633493,0.000000,0.000000,0.000000,0.000000,0.000000,0.541743,0.139575,0,1
7,1,1,6,6,6,6,2,1,20.679649,9.497786,1.223660,0.150382,0.000000,0.000000,0.000000,0.000000,0.576318,0.071071,1,0
8,1,1,22,21,18,15,13,10,66.691933,23.545543,6.151117,0.496372,0.000000,0.000000,0.000000,0.000000,0.500073,0.116793,0,1
9,1,1,79,75,73,71,64,47,22.141784,10.054384,0.874633,0.099780,0.023386,0.000000,0.000000,0.000000,0.560959,0.109134,0,1


### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames - one that contains all of the feature values and one that contains the class labels.

In [9]:
class_data = data['label']
#print(data['quality'].value_counts())
#print(data['prescreen'].value_counts())
feature_data = data.drop(['label', 'quality'] , axis = 1)
class_data = class_data.to_frame(name = "label")
labels = list(feature_data.columns)
print(class_data.head(7))
type(class_data)
print(feature_data.head(3))

   label
0      0
1      0
2      1
3      0
4      1
5      1
6      1
   prescreen  ma2  ma3  ma4  ma5  ma6  ma7   exudate8   exudate9  exudate10  \
0          1   22   22   22   19   18   14  49.895756  17.775994   5.270920   
1          1   24   24   22   18   16   13  57.709936  23.799994   3.325423   
2          1   62   60   59   54   47   33  55.831441  27.993933  12.687485   

   exudate11  exudate12  exudate13  exudate14  exudate15    euDist  diameter  \
0   0.771761   0.018632   0.006864   0.003923   0.003923  0.486903  0.100025   
1   0.234185   0.003903   0.003903   0.003903   0.003903  0.520908  0.144414   
2   4.852282   1.393889   0.373252   0.041817   0.007744  0.530904  0.128548   

   amfm_class  
0           1  
1           0  
2           0  


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). 

Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [10]:
standard_scale = StandardScaler()
scaler = standard_scale.fit(feature_data)
type(scaler)
scale_array = standard_scale.transform(feature_data)
#type(scale_array)
scale_frame = pd.DataFrame(data = scale_array , columns = labels)
#print(scale_frame.head())
type(scale_frame)

pandas.core.frame.DataFrame

Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [11]:
feature_train , feature_test , class_train , class_test = train_test_split( scale_frame, class_data , test_size = 0.20  , train_size = 0.80)
print("The number of rows in the training set is " + str(len(feature_train)) +  ".")
print("The number of rows in the testing set is " + str(len(feature_test)) + ".")

The number of rows in the training set is 920.
The number of rows in the testing set is 231.


Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [12]:
from sklearn.decomposition import PCA
components = [ 1, 2 , 3 , 4 ,5 ,6 , 7 , 8 , 9 , 10]
index = 0
var_sum = 0
num_cols = 0
while var_sum <= 0.95:
    pca = PCA(n_components = components[index])
    pca.fit(feature_train)
    var_sum = sum(pca.explained_variance_ratio_)
    num_cols = components[index]
    print("Columns kept = " , components[index])
    index = index + 1
    print("Variance retained = " , var_sum)
    print()

pca_use = PCA(n_components = num_cols)
pca_use.fit(feature_train)
train_pca = pca_use.transform(feature_train)
train_frame = pd.DataFrame( data = train_pca , columns = [ "pc1", "pc2" , "pc3" , "pc4" , "pc5" , "pc6" , "pc7" , "pc8"])
pca_use.fit(feature_test)
test_pca = pca_use.transform(feature_test)
test_frame = pd.DataFrame( data = test_pca , columns = [ "pc1", "pc2" , "pc3" , "pc4" , "pc5" , "pc6" , "pc7" , "pc8"])
print(test_frame.shape)
print(train_frame.shape)

Columns kept =  1
Variance retained =  0.3554714582793178

Columns kept =  2
Variance retained =  0.6153519118001274

Columns kept =  3
Variance retained =  0.7291384095058243

Columns kept =  4
Variance retained =  0.7954599777477191

Columns kept =  5
Variance retained =  0.8510759576259974

Columns kept =  6
Variance retained =  0.9000620511987278

Columns kept =  7
Variance retained =  0.9433245477144181

Columns kept =  8
Variance retained =  0.9671318518496339

(231, 8)
(920, 8)


### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [13]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy')
clf.fit(train_frame ,class_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [14]:
pred_labels = clf.predict(test_frame)
#print(pred_labels)
#print(len(pred_labels))
from sklearn.metrics import accuracy_score
print("The accuracy score is " + str(accuracy_score(class_test, pred_labels)))

The accuracy score is 0.5238095238095238


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [15]:
min_samples_spl = [ 5 , 10 , 15 , 20 , 25, 30 , 35 , 40 , 50  ] 
for sample in min_samples_spl:
    clf = DecisionTreeClassifier(criterion = 'entropy' , min_samples_split = sample)
    clf.fit(train_frame ,class_train)
    pred_labels_ms = clf.predict(test_frame)
    print( str(accuracy_score(class_test, pred_labels_ms)) , sample)  
#best number of min samples is 35 , acc score = .5021
print()
depths = [ 1, 2 , 3 ,4 , 5 ,6 , 7 , 8 , 9]
for depth in depths:
    clf = DecisionTreeClassifier(criterion = 'entropy' , max_depth = depth)
    clf.fit(train_frame ,class_train)
    pred_labels_dpt = clf.predict(test_frame)
    print( str(accuracy_score(class_test, pred_labels_dpt)) , depth)
#best depth = 5 , acc of .54
clf = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 5 , min_samples_split = 30)
clf.fit(train_frame ,class_train)
pred_labels_t1 = clf.predict(test_frame)
print( str(accuracy_score(class_test, pred_labels_t1)))

print()
imps = [ 0.007 , 0.008 , 0.009 , 0.01 ]
for imp in imps:
    clf = DecisionTreeClassifier(criterion = 'entropy' , min_impurity_decrease = imp)
    clf.fit(train_frame ,class_train)
    pred_labels_imp = clf.predict(test_frame)
    print( str(accuracy_score(class_test, pred_labels_imp)) , imp)
    
clf = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 5 , min_samples_split = 35, min_impurity_decrease = .0085)
clf.fit(train_frame ,class_train)
pred_labels_t2 = clf.predict(test_frame)
print( str(accuracy_score(class_test, pred_labels_t2)))

0.5064935064935064 5
0.5194805194805194 10
0.5064935064935064 15
0.5108225108225108 20
0.5454545454545454 25
0.5281385281385281 30
0.5194805194805194 35
0.5194805194805194 40
0.5064935064935064 50

0.47619047619047616 1
0.5367965367965368 2
0.5670995670995671 3
0.5757575757575758 4
0.5584415584415584 5
0.5800865800865801 6
0.5411255411255411 7
0.5367965367965368 8
0.5454545454545454 9
0.5497835497835498

0.5411255411255411 0.007
0.5714285714285714 0.008
0.5757575757575758 0.009
0.5714285714285714 0.01
0.5497835497835498


### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q7. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the data before it underwent PCA. So you want to use the standardized data that came out of Q2. 

In [16]:
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 5, min_impurity_decrease =.0085 )
scores = cross_val_score(clf, scale_frame,class_data , cv=10) 
print("accuracy = " + str(scores.mean() * 100))


accuracy = 62.638680659670165


Q8. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [17]:
from sklearn.model_selection import GridSearchCV

params = {"max_depth": [5,10,15,20], 
         "min_samples_leaf":  [5,10,15,20],
         "max_features" : [5 , 10 ,15]}

search = GridSearchCV(clf, params, cv=5, scoring='accuracy')
search.fit(scale_frame , class_data)
print(search.best_params_)
print("Accuracy:", search.best_score_*100)

{'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 20}
Accuracy: 64.63944396177237


Q9. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [18]:
#folds = [ 3 , 4 , 5 , 6 , 7 ,8 ]
#for fold in folds:
#5 appears to be the best number of folds
nested_score =  cross_val_score(search, scale_frame, class_data, cv= 5)
print("Accuracy:", nested_score.mean()*100 , " for 5 folds.")

Accuracy: 63.4240542066629  for 5 folds.


Q10. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What can we do to improve the accuracy of the model? Answer as a comment:

In [19]:
#One idea for how to improve the model is through pre-pruning the tree using a statistical test. Using a 
#chisquared test is the most obvious first step to take as this measures whether or not a spilit is statistically
#significant. Other statistical tests could also be examined in order to see if they provide a better accuracy score.
#Another idea is to post prune the tree, most likely through reduced error pruning which will allow the model to compare 
#the results from the validation set and training set in order to determine if the split chosen is beneficial to the model.
#If neither pre nor post pruning deliver better results, then then next step would be to try some sort of ensemble method in
#order to see if taking a different approach in creating the decision tree will result in a more accurate tree. If all of the 
#above are tried and still no improvement is made, then it may be that this data set cannot be accurately represented by 
#decision trees in the way that is needed.